<h1 style="font-size:50px; text-align:center">DCO Detections</h1>

In [15]:
import h5py as h5
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import seaborn as sns
import legwork
import os
import sys
sys.path.append("../src/")
from variations import variations
from galaxy import simulate_mw

In [16]:
%config InlineBackend.figure_format = 'retina'
plt.rc('font', family='serif')
fs = 20

params = {'legend.fontsize': fs,
         'axes.labelsize': fs,
         'xtick.labelsize':0.7*fs,
         'ytick.labelsize':0.7*fs}
plt.rcParams.update(params)

In [17]:
dco_colours = {"BHBH": plt.get_cmap("plasma")(0.2), 
               "BHNS": plt.get_cmap("plasma")(0.5),
               "NSNS": plt.get_cmap("plasma")(0.8)}

dco_types = ["BHBH", "BHNS", "NSNS"]
data_path = "../data/"
sim_folder = data_path + "simulation/"

total_DCOs_in_MW = np.load(data_path + "total_DCO_in_MW.npy")
total_bound_DCOs_in_MW = np.load(data_path + "total_DCO_in_MW_nohubble.npy")

# Calculate number of detections

In [12]:
def get_detections(dco_type, MW_SIZE=200000, MW_MASS_FAC=1.0, t_obs=4*u.yr, only_frac=False):
    # set up arrays for return
    detections = np.zeros(shape=(len(dco_types), len(variations), 2500))
    
    # go through each physics variation
#     for d in range(len(dco_types)):
    
    for d in range(len(dco_types)):
        for v in range(len(variations)):
            # open the proper output file
            fname = sim_folder + "{}_{}_all.h5".format(dco_types[d], variations[v]["file"])
            if os.path.isfile(fname):
                with h5.File(fname, "r") as f:
                    parameter = "n_detect"
                    if v in [7, 8, 9, 17, 19]:
                        parameter = "n_ten_year"
                    n_detect = f["simulation"].attrs[parameter].astype(np.int)
                    total_mw_weight = f["simulation"].attrs["total_MW_weight"]
                    full_data = f["simulation"][...].squeeze()
                
#                 circ_sources = legwork.source.Source(m_1=full_data["m_1"] * u.Msun, m_2=full_data["m_2"] * u.Msun,
#                                            dist=full_data["dist"] * u.kpc, ecc=np.zeros(len(full_data["m_1"])),
#                                            a=full_data["a_LISA"] * u.AU)
#                 circ_snr = circ_sources.get_snr()
                
#                 print(dco_types[d])
#                 true_rate = np.sum(full_data["weight"][full_data["snr"] >= 7]) / np.sum(total_mw_weight) * total_DCOs_in_MW[d][v] * MW_MASS_FAC
#                 circ_rate = np.sum(full_data["weight"][circ_snr >= 7]) / np.sum(total_mw_weight) * total_DCOs_in_MW[d][v] * MW_MASS_FAC
#                 print(true_rate, circ_rate, (true_rate - circ_rate) / true_rate)
#                 print()

                # go through the file and normalise the detections for all binaries with SNR > x
                cursor = 0
                detections_per_MW = np.zeros(len(n_detect))
                
                checks = np.zeros(len(n_detect))
                for i in range(len(n_detect)):
                    snr = full_data["snr"][cursor:cursor + n_detect[i]]
                    weights = full_data["weight"][cursor:cursor + n_detect[i]]
                    checks = np.sum(weights)**2 / np.sum(weights**2)
                    detections[d][v][i] = np.sum(weights[snr * np.sqrt(t_obs / (4 * u.yr)) > 7]) / total_mw_weight[i]
                    cursor += n_detect[i]
                if not only_frac:
                    detections[d][v] *= total_DCOs_in_MW[d][v] * MW_MASS_FAC

            # set to zero if no file exists (simulation will have crashed)
            else:
                detections[d][v] = 0
                
            print(detections[d][v])
    
    return detections

In [5]:
def get_detections_alt(dco_type, MW_SIZE=100000, MW_MASS_FAC=1.5, t_obs=4*u.yr, only_frac=False):
    # set up arrays for return
    detections = np.zeros(shape=(len(dco_types), len(variations), 2500))
    
    fname = sim_folder + "{}_{}_all.h5".format(dco_types[dco_type], variations[0]["file"])
    if os.path.isfile(fname):
        with h5.File(fname, "r") as f:
            total_mw_weight = f["simulation"].attrs["total_MW_weight"].sum()
            full_data = f["simulation"][...].squeeze()
            
        factor = total_DCOs_in_MW[dco_type][0] * MW_MASS_FAC / total_mw_weight
        
        weights = full_data["weight"][full_data["snr"] > 7]
        avg = weights.sum() * factor
        print(avg)
        
    sample_rates = np.zeros(1000)
    for i in range(len(sample_rates)):
        sample = np.random.choice(weights, len(weights), replace=True)
        sample_rates[i] = sample.sum() * factor
        
    print(np.percentile(sample_rates, [5, 95]))
    
    sample_rates = np.zeros(1000)
    for i in range(len(sample_rates)):
        sample = np.random.choice(weights, 100, replace=True)
        sample_rates[i] = sample.sum() * factor * (len(weights) / 100)
        
    print(np.percentile(sample_rates, [5, 95]))

In [13]:
detections_4yr = get_detections(dco_types)

[188.53132612  92.34068491 110.65467279 ...  76.09741935 113.23238329
  84.43272143]
[ 44.39526204  87.90986848  65.62856389 ...  94.37928739  88.88728249
 120.32520638]
[56.52694073 59.05925402 66.61515288 ... 67.86646924 53.60359513
 83.49439941]
[ 78.66441656 104.46929416 100.72448952 ...  37.77374984  93.64765327
  87.46964701]
[ 72.43467848  86.76929771  32.41913152 ...  85.33662976 103.54711038
  55.5299427 ]
[261.0843016  257.04690057 293.21462966 ...  88.46689663 139.4015524
 303.99706783]
[45.54866626 41.39222345 37.71110985 ... 38.97435545 36.20761162
 19.85184678]
[ 87.63420197  61.14160151 143.04181091 ... 161.52964088  78.51305133
 166.87207368]
[ 79.17846851  75.15629446 154.55851205 ...  90.70297393 145.18578219
  97.13841458]
[0. 0. 0. ... 0. 0. 0.]
[221.59229951  81.21770627 127.62432459 ... 315.62085411 186.42320579
 246.40210526]
[ 56.9636817   91.27605326  84.51157102 ...  64.44891843  86.58810645
 111.03212926]
[201.08948553 268.44151361  81.98711141 ... 121.703208

<ipython-input-12-955773ba0689>:40: RuntimeWarning: invalid value encountered in double_scalars
  checks = np.sum(weights)**2 / np.sum(weights**2)


[36.14185379 21.63848667 22.85295891 ... 34.33913528 36.71255205
 27.92619249]
[15.57677404  7.41507175 17.40544723 ... 12.7637186   8.67870967
 17.21095397]
[13.99659303  6.63974364 10.15455435 ... 16.42890068 16.90366061
  8.521889  ]
[15.33901952  8.9052546  21.5699419  ...  7.17932093  4.97526637
  7.10394811]


In [6]:
detections_10yr = get_detections(dco_types, t_obs=10*u.yr)

In [7]:
detections_only_fractions = get_detections(dco_types, only_frac=True)

In [14]:
np.save("../data/detections_4yr", detections_4yr)
np.save("../data/detections_10yr", detections_10yr)
np.save("../data/detections_only_fractions", detections_only_fractions)

NameError: name 'detections_10yr' is not defined

# What if we assumed everything was circular?

In [38]:
fid_sources = [None, None, None]
total_mw_weights = [None, None, None]
for i in range(len(dco_types)):
    with h5.File(sim_folder + "{}_massTransferEfficiencyFixed_0_5_all.h5".format(dco_types[i]), "r") as f:
        full_data = f["simulation"][...].squeeze()
        snr_mask = full_data["snr"] > 7
        
        data = full_data[snr_mask]
        
        total_mw_weights[i] = f["simulation"].attrs["total_MW_weight"].sum()
        
    fid_sources[i] = legwork.source.Source(m_1=data["m_1"] * u.Msun, m_2=data["m_2"] * u.Msun,
                                           dist=data["dist"] * u.kpc, ecc=data["e_LISA"],
                                           a=data["a_LISA"] * u.AU)
    fid_sources[i].weight = data["weight"]
    fid_sources[i].zams_mask = np.logical_and(data["m_1_ZAMS"] >= 20, data["m_1_ZAMS"] <= 120)

In [40]:
for i in range(len(dco_types)):
    snr_true = fid_sources[i].get_snr()
    new_source = fid_sources[i]
    new_source.ecc = np.zeros(new_source.n_sources)
    snr_circ = new_source.get_snr()
    
    full_mask = np.logical_and(fid_sources[i].zams_mask, snr_circ > 7)
    
    print(np.sum(new_source.weight[full_mask]) / total_mw_weights[i] * total_DCOs_in_MW[i][2])

40.60807861360013
10.22671235981745
0.1015822905330661
